In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [25]:
#Load an Excel file
def load_data(path):
    path = Path(path)
    return pd.read_excel(path)

In [27]:
#Save the enriched DataFrame to CSV without index
def save_enriched_data(df, output_file):
    output_file = Path(output_file)
    df.to_csv(output_file, index=False)

In [29]:
#Create 'total_nights' = stays_in_week_nights + stays_in_weekend_nights
def compute_total_nights(df):
    df = df.copy()
    df["total_nights"] = df["stays_in_week_nights"] + df["stays_in_weekend_nights"]
    return df

In [31]:
#Infer a coarse 'season' from 'arrival_date_month':
#Summer: June, July, August
#Shoulder: May, September
#Off-peak: others
def infer_season(df):
    df = df.copy()
    month = df.get("arrival_date_month")
    m = pd.Series(month, index=df.index).astype(str).str.title() 
    summer = {"June", "July", "August"}
    shoulder = {"May", "September"}
    df["season"] = np.where(m.isin(summer), "Summer",
                      np.where(m.isin(shoulder), "Shoulder", "Off-peak"))
    return df

In [33]:
#Derive:
#child_count: numeric children count
#breakfast_pax: number of people charged breakfast if meal plan includes breakfast(meal in {"BB","HB","FB"} → has breakfast)

def derive_breakfast_and_children(df):
    df = df.copy()
    df["child_count"] = pd.Series(df.get("children", 0)).fillna(0).astype(float)
    adults = pd.Series(df.get("adults", 0)).fillna(0).astype(float)
    children = df["child_count"]
    meal = df.get("meal", "")
    has_breakfast = pd.Series(False, index=df.index)
    if isinstance(meal, pd.Series):
        has_breakfast = meal.isin(["BB", "HB", "FB"])
    df["breakfast_pax"] = np.where(has_breakfast, adults + children, 0.0)
    return df

In [35]:
#Compute 'adjusted_revenue' = adr * total_nights * (1 - is_canceled).
def compute_adjusted_revenue(df):
    df = df.copy()
    valid = (df["adr"] >= 0) & (df["total_nights"] >= 0)
    df["adjusted_revenue"] = df["adr"] * df["total_nights"] * (1 - df["is_canceled"])
    df.loc[~valid, "adjusted_revenue"] = np.nan
    return df

In [37]:
"""
Compute per-night and total costs using simple embedded rules:
    - Base variable cost per night = 60
    - Cleaning cost per stay (fixed) = 30
    - Child cost per child per night = 20
    - Breakfast cost per person per night = 6
    - Season uplifts on cost: Summer +20%, Shoulder +10%, Off-peak +0%
    Output: cost_per_night_adj, total_cost
"""
def add_costs(df):
    df = df.copy()

    # Embedded constants
    base_cost_per_night = 60.0
    cleaning_cost_per_stay = 30.0
    child_cost_per_night = 20.0
    breakfast_unit_cost = 6.0
    charge_fixed_if_cancelled = False

    season_cost_uplift = {"Summer": 0.20, "Shoulder": 0.10, "Off-peak": 0.00}

    season_up = pd.Series(df["season"]).map(season_cost_uplift).fillna(0.0)

    df["cost_per_night_adj"] = (
        base_cost_per_night * (1 + season_up) 
        + pd.Series(df.get("child_count", 0)).fillna(0) * child_cost_per_night
        + pd.Series(df.get("breakfast_pax", 0)).fillna(0) * breakfast_unit_cost
    )

    if charge_fixed_if_cancelled:
        fixed = cleaning_cost_per_stay
    else:
        fixed = np.where(df["is_canceled"] == 1, 0.0, cleaning_cost_per_stay)

    df["total_cost"] = pd.Series(df["total_nights"]).fillna(0) * df["cost_per_night_adj"] + fixed
    return df

In [39]:
'''
profit = adjusted_revenue - total_cost
Booking_Profitability_Score: scale profit into [0,100] by clipping to 5th–95th pct
score = 100 * (clip(profit) - p5) / (p95 - p5)
If all profits equal (p5≈p95), set score=50 for non-NaN profits
'''
def compute_profit_and_score(df):
    df = df.copy()
    df["profit"] = df["adjusted_revenue"] - df["total_cost"]

    prof = df["profit"].dropna()
    if prof.empty:
        df["profit_clip"] = np.nan
        df["Booking_Profitability_Score"] = np.nan
        return df

    p5, p95 = np.nanpercentile(prof, [5, 95])
    if np.isclose(p5, p95):
        df["profit_clip"] = df["profit"]
        df["Booking_Profitability_Score"] = 50.0
        return df

    df["profit_clip"] = df["profit"].clip(lower=p5, upper=p95)
    df["Booking_Profitability_Score"] = 100 * (df["profit_clip"] - p5) / (p95 - p5)
    return df


In [41]:
'''
End-to-end pipeline:
load -> build features -> adjusted revenue -> costs -> profit & score -> save.
'''
def main(input_file, output_file):
    df = load_data(input_file)
    df = compute_total_nights(df)
    df = infer_season(df)
    df = derive_breakfast_and_children(df)

    df = compute_adjusted_revenue(df)
    df = add_costs(df)
    df = compute_profit_and_score(df)

    save_enriched_data(df, output_file)
    print(f"\nSaved → {Path(output_file).resolve()}")

    return df

In [43]:
if __name__ == "__main__":
    input_path = "/Users/xiafeier/Desktop/python/hotel_bookings.xlsx"  
    output_path = "hotel_bookings_enriched.csv"
    final_df = main(input_path, output_path)


Saved → /Users/xiafeier/hotel_bookings_enriched.csv
